# Indeed

In [ ]:
! pip install selenium

In [ ]:
# https://www.swtestacademy.com/install-chrome-driver-on-mac/
# https://janakiev.com/blog/jupyter-virtual-envs/

In [112]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException

import time
import numpy as np
import pandas as pd
import re

In [147]:
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()

# Write your keyword's jobs
inputElement_what = browser.find_element_by_xpath('//*[@id="text-input-what"]')
inputElement_what.send_keys('Data Scientist')
time.sleep(0.30)

# Write your localisation
inputElement_where = browser.find_element_by_xpath('//*[@id="text-input-where"]')
inputElement_where.send_keys('Île-de-France')
time.sleep(0.30)

# click submit button
submit_button = browser.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div/div/div/form/div[3]/button')[0]
submit_button.click()
time.sleep(0.30)

In [114]:
# Aller d'annonce à annonce (go and back)
list_job = browser.find_elements_by_class_name('title')
len(list_job)


15

In [123]:
elems = browser.find_elements_by_css_selector("[data-tn-element='jobTitle']")
print(elems)
liste_element = [elem.get_attribute(name="href") for elem in elems]
print(liste_element)

[<selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="8156cb21-f533-4f7d-a810-a1ed8f08ec04")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="606ce3b4-e31f-4ffb-a1b4-e9e832fd2c5e")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="f7287ae4-f57e-44b3-b80c-74ef5122f8dc")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="be58af51-c330-461b-8021-a00349f6dab1")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="5cfbdff4-d1ac-473a-b50d-e929fc6db8f7")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="39fca76f-4a25-4272-a04e-925728c88cf2")>, <selenium.webdriver.remote.webelement.WebElement (session="a4e83fc3e88cb215cefb04f7002125ae", element="eab7b737-3b1c-4cb5-ada0-5e4b4b67

In [ ]:
# Aller d'annonce à annonce (go back) et page en page
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()

# Write your keyword's jobs
inputElement_what = browser.find_element_by_xpath('//*[@id="text-input-what"]')
inputElement_what.send_keys('Data Scientist')
time.sleep(0.30)

# Write your localisation
inputElement_where = browser.find_element_by_xpath('//*[@id="text-input-where"]')
inputElement_where.send_keys('Île-de-France')
time.sleep(0.30)

# click submit button
submit_button = browser.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div/div/div/form/div[3]/button')[0]
submit_button.click()
time.sleep(0.30)

page = True

while page:    
    list_job = browser.find_elements_by_class_name('title')
    for i in range(len(list_job)):
        list_job = browser.find_elements_by_class_name('title')
        if i <= len(list_job):
            actions = ActionChains(browser)
            browser.execute_script("arguments[0].scrollIntoView();", list_job[i])
            time.sleep(1) 
            actions.perform()
            #if list_job[i] == []:
            try: 
                list_job[i].click()
                time.sleep(1)
                browser.back()
            except ElementClickInterceptedException:
                pop_up = browser.find_element_by_css_selector('#popover-x > a > svg > g > path')
                pop_up.click()
                time.sleep(3)

    try:
        next_page = browser.find_element_by_css_selector('#resultsCol > nav > div > ul > li:nth-child(7) > a > span > span > svg')
        actions = ActionChains(browser)
        actions.move_to_element(next_page)
        time.sleep(3)
        actions.perform()
        next_page.click()
        time.sleep(3)
        #browser.back()

    except NoSuchElementException:
        try: 
            next_page = browser.find_element_by_css_selector('#resultsCol > nav > div > ul > li:nth-child(6) > a > span > span > svg')
            print(next_page)
            actions = ActionChains(browser)
            actions.move_to_element(next_page)
            time.sleep(3)
            actions.perform()
            next_page.click()
            time.sleep(3)
            #browser.back()

        except ElementClickInterceptedException:
            pop_up = browser.find_element_by_css_selector('#popover-x > a > svg > g > path')
            pop_up.click()
            time.sleep(3)

    except:
        page = False
        

In [ ]:
# Pop-up
pop_up = browser.find_element_by_css_selector('#popover-x > a > svg > g > path')
pop_up.click()

In [87]:
# Récuperer les informations par annonce

d_xpath = {'title':'.//h2[@class="title"]//a',
       'companies':'.//span[@class="company"]',
       'localisation':'.//span[contains(@class,"location")]',
       'links':'.//h2[@class="title"]//a', 
       'review':'.//span[@class="ratingsContent"]',
       'salary':'.//span[@class="salaryText"]',
       'descriptions':'//div[@id="jobDescriptionText"]'
       }

def update_dic(browser, dic, column, d, name= None):
    """
    Met à jour le dictionnaire d à chaque fiche (cf boucle while plus bas)
    dic: dictionnaire, soit d_id, d_class_name
    column: nom de la colonne de la dataframe et de la clé du dictionnaire
    """
    if dic == d_xpath:
        try:
            d[column] = browser.find_element_by_xpath(dic[column]).text
        except:
            d[column] = browser.find_element_by_xpath(dic[column]).get_attribute(name= name)
        except NoSuchElementException:
            d[column] = 'Non reference'
    return d

def returninfojob(d):
    """
    Retourne un dictionnaire contenant toute les informations nécessaires pour les offres d'emploi
    """
    d = update_dic(job, d_xpath,'title', d, name = "title")
    d = update_dic(job, d_xpath,'companies', d)
    d = update_dic(job, d_xpath,'localisation', d)
    d = update_dic(job, d_xpath,'links', d, name = "href")
    d = update_dic(job, d_xpath,'review', d)
    d = update_dic(job, d_xpath,'salary', d)
    d = update_dic(job, d_xpath,'descriptions', d)
    d = update_dic(job, d_xpath,'date', d)
    return d

In [88]:
df_indeed = pd.DataFrame(columns=['title', 'companies','localisation','links','review','salary', 'descriptions', 'date'])

In [41]:
# Aller d'annonce à annonce (go back) et page en page
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()

# Write your keyword's jobs
inputElement_what = browser.find_element_by_xpath('//*[@id="text-input-what"]')
inputElement_what.send_keys('Data Scientist')
time.sleep(0.30)

# Write your localisation
inputElement_where = browser.find_element_by_xpath('//*[@id="text-input-where"]')
inputElement_where.send_keys('Île-de-France')
time.sleep(0.30)

# click submit button
submit_button = browser.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div/div/div/form/div[3]/button')[0]
submit_button.click()
time.sleep(0.30)

page = True

while page:  
    list_job = browser.find_elements_by_class_name('title')
    for i in range(len(list_job)):
        list_job = browser.find_elements_by_class_name('title')
        if i <= len(list_job):
            actions = ActionChains(browser)
            browser.execute_script("arguments[0].scrollIntoView();", list_job[i])
            time.sleep(1) 
            actions.perform()
            #if list_job[i] == []:
            try: 
                list_job[i].click()
                d = {}
                d = returninfojob(d)
                df_indeed = df_indeed.append(d, ignore_index=True)
                print('ok')
                time.sleep(1)
                browser.back()
            except ElementClickInterceptedException:
                pop_up = browser.find_element_by_css_selector('#popover-x > a > svg > g > path')
                pop_up.click()
                time.sleep(3)

    try:
        next_page = browser.find_element_by_css_selector('#resultsCol > nav > div > ul > li:nth-child(7) > a > span > span > svg')
        actions = ActionChains(browser)
        actions.move_to_element(next_page)
        time.sleep(3)
        actions.perform()
        next_page.click()
        time.sleep(3)
        #browser.back()

    except NoSuchElementException:
        try: 
            next_page = browser.find_element_by_css_selector('#resultsCol > nav > div > ul > li:nth-child(6) > a > span > span > svg')
            print(next_page)
            actions = ActionChains(browser)
            actions.move_to_element(next_page)
            time.sleep(3)
            actions.perform()
            next_page.click()
            time.sleep(3)
            #browser.back()

        except ElementClickInterceptedException:
            pop_up = browser.find_element_by_css_selector('#popover-x > a > svg > g > path')
            pop_up.click()
            time.sleep(3)

    except:
        page = False
        

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
<selenium.webdriver.remote.webelement.WebElement (session="9e95cf18258b6888bf076e1348ed9186", element="a306fb82-76e7-4ac1-961a-18949f0dee9f")>
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#resultsCol > nav > div > ul > li:nth-child(6) > a > span > span > svg"}
  (Session info: chrome=85.0.4183.83)


In [42]:
df_indeed

,title,societe,localisation,info,date
0,Non reference,Non reference,Non reference,Non reference,Non reference
1,Non reference,Non reference,Non reference,Non reference,Non reference
2,Non reference,Non reference,Non reference,Non reference,Non reference
3,Non reference,Non reference,Non reference,Non reference,Non reference
4,Non reference,Non reference,Non reference,Non reference,Non reference
...,...,...,...,...,...
439,Non reference,Non reference,Non reference,Non reference,Non reference
440,Non reference,Non reference,Non reference,Non reference,Non reference
441,Non reference,Non reference,Non reference,Non reference,Non reference
442,Non reference,Non reference,Non reference,Non reference,Non reference


In [111]:
# Aller d'annonce à annonce (go back) et page en page
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()

# Write your keyword's jobs
inputElement_what = browser.find_element_by_xpath('//*[@id="text-input-what"]')
inputElement_what.send_keys('Data Scientist')
time.sleep(0.30)

# Write your localisation
inputElement_where = browser.find_element_by_xpath('//*[@id="text-input-where"]')
inputElement_where.send_keys('Île-de-France')
time.sleep(0.30)

# click submit button
submit_button = browser.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div/div/div/form/div[3]/button')[0]
submit_button.click()
time.sleep(0.30)

page = True

while page:    
    list_job = browser.find_elements_by_class_name('title')
    for i in range(len(list_job)):
        job_card = browser.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
        for job in job_card:
            actions = ActionChains(browser)
            browser.execute_script("arguments[0].scrollIntoView();", job)
            time.sleep(1) 
            actions.perform()
            job.click()
            try: 
                title_job =  job.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
                print(title_job)
            except:
                links = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href")
                for link in links:
                    browser.get(link)
                    description = browser.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
                    print('description')
            time.sleep(1)
            browser.back()


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=85.0.4183.83)


In [99]:
job_card = browser.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
job_card
for job in job_card:
    title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
    print(title)

DATA SCIENTIST H/F - CDI - ASNIERES (92)
Data Analyst - Business Intelligence Junior H/F
Data Business Analyst H/F
Data Scientist (F/H)
Développeur Python / Data scientist H/F – CDI - H/F
DATA SCIENTIST - F/H
Data Scientist
Data Scientist H/F
Data Scientist F/H
CDD - Data scientist junior H/F
Data scientist F/H
DATA SCIENTIST - IMAGERY
DATA ANALYST – CDI (H/F)
Consultant Data scientist F/H
Technical Data Analyst (F/H)
Senior Data Scientist H/F - CDI (Levallois/Corte)


In [48]:
def get_liste_link():
    """
    Retourne la liste de tout les objets selenium correspondant aux liens pour accéder aux annonces
    """
    elems = browser.find_elements_by_xpath("//a[@data-tn-element='jobTitle']")
    tab_url = [elem.get_attribute("href") for elem in elems]
    return tab_url

In [51]:
url = get_liste_link()
url

['https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0ClbwjSiU8j4rtOmxlRXLUWR0Zsol6jBDzIj5LytFiCin5odrsRgFGQkw_5tcCdZlpx3Fu0aUZgfhv0g3lQ5Os5SttIwa2uUgIaajh2RmfscRRjj3iBqFEjZkosIypNauy4X7EdRL4EQFV2_ZhB_DzyEVq6sPoVO1vdZj6rNYgo6-GOIFENxsSmx4WxavjaybPNnZxx722BSBdDebLZE_W5goOdKWXMdZqRaEaz9yI-ZPnbpErQOy_rFBiF-icgAMwboWtu5RZa7MmZlZxhPAeTFJWU6DmFixlzmaBqn7Yn1xZqhDTPxaBtqqr0OuCGp1FQBHRNzpRMvJUvCxd4rj0meoQDuvHPckf4BB-js8cDF37_o9ddIK91ExERxEllIBn8DA2hE3oTkXlVkwqQySS4yp1hAOD60BwkUxclKgkB04Tc5SNtQiTxC4LUsYwkhWqboajPyyGxzDmyifVO9aev3E7Li8aCc2AIzRAYS7-f5Xh3G5UBHsHN4MPkZsYF-DTub9wWdoQQ8Q==&p=0&fvj=1&vjs=3',
 'https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0ClbwjSiU8j4rtOmxlRXLUWR0Zsol6jBDzIj5LytFiCin5odrsRgFGQkw_5tcCdZlq-lLvxu7FzP811AwajBadPNnwRhhfpmK2_G8RP-uspj4c2Sts6Nn0stm03bkcsZUNqvVMwabB35HYB65VYPp1U6S8fyQtqu_Esh-xGLIsrYxmb4hxFy4c35Ol_LAuhZQEAacaQYk6iJchAHEil9a22kF8CEyhmF3mvb6p4qmH5i3gNgNYfE7tML0BNP4wKZihcK2LvAQRKLdOwkOtF2MN1hdJ_f7OslIyO9dxi7ZrTZumx-ObmNicnIP6UxSpPipD-yT6a2wEVqSy8z2kn1GRk12p_DZg

In [124]:
onejob='https://www.indeed.fr/voir-emploi?cmp=BCA-Expertise&t=Data+Scientist&jk=891a01a98029da69&sjdu=QwrRXKrqZ3CNX5W-O9jEvQGG825vj9bF1q_xBb9G7pwKh6gTpTQMqaILbNsrCW96IJiojGEK1kBPUdU25somk7CvgGt1SMqkpO5kqYVyj2s&adid=331164141&ad=-6NYlbfkN0ClbwjSiU8j4rtOmxlRXLUWR0Zsol6jBDzIj5LytFiCin5odrsRgFGQkw_5tcCdZlpx3Fu0aUZgfhv0g3lQ5Os5SttIwa2uUgIaajh2RmfscRRjj3iBqFEjpjvAWXhmLP9zT3WDZYdNK0edEpGTbBB20-2vomv9Cz1YlaAmTGk6_w42OipNE_KUAUPbL1Bv7xNIXGC6NTgipSNQn7CNCaah76L8A2m81oqXmk6iWAnzP1EMv1lBFpSb12BsFxh8R8lRP_htRpD0RAfBC2u78z2iPva7YS2rn2We6erR5uWJfyl_Es2EIOiw&vjs=3'

In [127]:
browser = webdriver.Chrome()
browser.get(onejob)
browser.maximize_window()

In [141]:
title_job =  browser.find_element_by_xpath('//div[contains(@class,"jobsearch-InlineCompanyRating")]/div[4]').text

In [142]:
title_job

'Asnières-sur-Seine (92)'

In [148]:
next_page = browser.find_element_by_xpath("//a[@aria-label='Suivant'] | a[contains(text(),'Suivant')] ").get_attribute(name="href")
next_page

'https://www.indeed.fr/jobs?q=Data+Scientist&l=Sannois+%2895%29%C3%8Ele-de-France&start=10'